In [141]:
import os

In [142]:
%pwd

'f:\\Time Series Data Analysis (Upwork)\\end-to-end-time-series-data-analysis-with-mlflow'

In [143]:
#os.chdir('../')

In [144]:
%pwd

'f:\\Time Series Data Analysis (Upwork)\\end-to-end-time-series-data-analysis-with-mlflow'

In [145]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    # INPUT_SHAPE: list 
    # BATCH_SIZE: int
    # EPOCHS: int
    #CLASSES: int

In [146]:
from TimeSeriesProject .constants import *
from TimeSeriesProject.utils.common import read_yaml, create_directories

In [147]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            #params_input_size=self.params.INPUT_SHAPE,
            #params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [148]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [149]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    def get_base_model(self):
        #self.model = tf.keras.models.Sequential(
        self.model = Sequential()
        self.model.add(LSTM(64, input_shape=(9,1)))
            # tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(9, 1)),
            # tf.keras.layers.MaxPooling1D(pool_size=2),
            # tf.keras.layers.Flatten(),
            # tf.keras.layers.Dense(64, activation='relu')
        

        self.save_model(path=self.config.base_model_path, model=self.model)


    @staticmethod
    def _prepare_full_model(model, classes):
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="sigmoid"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.binary_crossentropy,
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=1
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [150]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-09-13 20:53:46,035: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-13 20:53:46,035: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-13 20:53:46,051: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-13 20:53:46,051: INFO: common: created directory at: artifacts]
[2023-09-13 20:53:46,051: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-09-13 20:53:46,467: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4_input (InputLayer)   [(None, 9, 1)]            0         
                                                                 
 lstm_4 (LSTM)               (None, 64)                16896     
                            